In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
from time import sleep
from threading import Thread

In [ ]:
class particle:
    
    
    def rotMat(alpha, x):
        return np.array([[np.cos(alpha), -np.sin(alpha)], [np.sin(alpha), np.cos(alpha)]]).dot(x)
    
    def __init__(self, m, x,y, vx, vy,r):
        self.m = m     #kg
        self.vx = vx   #m/s
        self.vy = vy   #m/s
        self.x = x     #m
        self.y = y     #m
        self.r = r     #m
        
        self.ratio = 1
    
    def move(self, dt, ratio):
            self.x = self.vx * ratio * dt+ self.x
            self.y = self.vy * ratio * dt + self.y
            self.ratio = ratio
            
    def bounceWall(self,n):
        match n:
            case 0:
                self.vx = -self.vx
                self.x = 2*self.r - self.x
            case 1:
                self.vx = -self.vx
                self.x= 2*(simulation.width.value*10**-12-self.r)-self.x
            case 2:
                self.vy = -self.vy
                self.y = 2*self.r-self.y
            case 3:
                self.vy = -self.vy
                self.y = 2*(simulation.height-self.r)-self.y
                
    def bounce(self,p):
        dx = self.x-p.x
        dy = -(self.y-p.y)
        d = np.sqrt(dx**2+dy**2)
        alpha = np.arctan2(dy,dx)
        
        p1=np.array([self.x,self.y])
        p2=np.array([p.x,p.y])
        p1s=particle.rotMat(alpha,p1)
        p2s=particle.rotMat(alpha,p2)
        p1ns=np.array([p1s[0]+(2*self.r-d)/2,p1s[1]])
        p2ns=np.array([p2s[0]-(2*self.r-d)/2,p2s[1]])
        p1n=particle.rotMat(-alpha,p1ns)
        p2n=particle.rotMat(-alpha,p2ns)
        self.x=p1n[0]
        self.y=p1n[1]
        p.x=p2n[0]
        p.y=p2n[1]
         
        v1 = np.array([self.vx, self.vy])
        v2 = np.array([p.vx, p.vy])
        v1s = particle.rotMat(alpha,v1)
        v2s = particle.rotMat(alpha,v2)
        v1ns=(v2s[0],v1s[1])
        v2ns=(v1s[0],v2s[1])
        v1n= particle.rotMat(-alpha,v1ns)
        v2n= particle.rotMat(-alpha,v2ns)
        self.vx = v1n[0]
        self.vy = v1n[1]
        p.vx = v2n[0]
        p.vy = v2n[1]
        
    def getKineticEnergy(self,dt):
        return 1/2 * self.m * ((self.vx*self.ratio)**2+(self.vy*self.ratio)**2)

In [ ]:
class simulation:
    
    p = []
    height = 3000e-12        #m (3000pm)
    dt = 20e-15              #s (200fs)
    radius = 50e-12          #m (50pm)
    k = 1.380649 *10**-23    #J/K
    impulse = np.zeros(200)
    
    startSimulation=False
    
    veloRatio = widgets.FloatSlider(min=0.1,max=3, value=1,step=0.01, description="Velocity")
    width = widgets.IntSlider(min=500, max= 8000, value=4000, description="Width (pm) ")
    buttonAdd = widgets.Button(description="+")
    buttonRem = widgets.Button(description="-")
    buttonStart = widgets.Button(description="start")
    buttonStop = widgets.Button(description="stop")
    textVolume = widgets.FloatText(description="Volume (pm³)", disabled=True)
    textPressure = widgets.FloatText(description="Pressure", disabled=True)
    textTemperature = widgets.FloatText(description="Temperature", disabled=True)
    textParticles = widgets.IntText(disabled = True)
    canvas = Canvas(width = width.max/10, height=height*10**12/10)
    
    def generateParticles(button=widgets.Button(), n=10):
        for _ in range(n):
            simulation.p.append(particle(2.6569e-26,random.randint(0,simulation.width.value)*10**-12,
                random.randint(0,simulation.height*10**12)*10**-12,
                random.randint(-1500,1500),random.randint(-1500,1500),50*10**-12))
            simulation.textParticles.value = len(simulation.p)
    
    def removeParticles(button=widgets.Button(), n=10):
            for _ in range(n):
                if simulation.p:
                    simulation.p.pop()
            simulation.textParticles.value = len(simulation.p)

    def deleteParticles():
        simulation.p = []
    
    def addParticle(particle):
        simulation.p.append(particle)
        
    def start(button=widgets.Button()):
        simulation.startSimulation = True
        t = Thread(target=simulation.animateParticles, daemon=True)
        t.start()
        
    def stop(button=widgets.Button()):
        simulation.startSimulation = False
        

    def collisionTest():
        count = 1
        for i in simulation.p:
            
            for j in simulation.p[count::]:
                dx = j.x-i.x
                dy = j.y-i.y
                if dx**2+dy**2 < (i.r+j.r)**2:
                     if (dx*i.vx <=0 and dx*j.vx >=0 ) and (dy*i.vy <=0 and dy*j.vy >=0 ):
                        pass 
                     else:
                        i.bounce(j)
            count = count + 1
            
            if i.x < i.r:
                i.bounceWall(0)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vx * i.ratio)
            elif i.x > simulation.width.value*10**-12-i.r:
                i.bounceWall(1)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vx * i.ratio)
                
            if i.y < i.r:
                i.bounceWall(2)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vy * i.ratio)
            elif i.y > simulation.height-i.r:
                i.bounceWall(3)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vy * i.ratio)
            
            
            
    
                                                
                
    def animateParticles():     #1px = 10pm
        while simulation.startSimulation:
            with hold_canvas():
                simulation.canvas.clear()
                simulation.canvas.stroke_rect(0,0,width=simulation.width.value/10,height=simulation.height*10**12/10)
                for i in simulation.p:
                    simulation.canvas.fill_circle(i.x*10**12/10, i.y*10**12/10, i.r*10**12/10)
                    i.move(simulation.dt, simulation.veloRatio.value)
            
            simulation.collisionTest()
            simulation.updateValues()
            sleep(simulation.dt*10**12)

    def initialize():
        simulation.buttonAdd.on_click(simulation.generateParticles)
        simulation.buttonRem.on_click(simulation.removeParticles)
        simulation.buttonStart.on_click(simulation.start)
        simulation.buttonStop.on_click(simulation.stop)
        
        items = [widgets.HBox([simulation.buttonAdd, simulation.buttonRem, simulation.textParticles]),
                 simulation.textVolume, simulation.buttonStart, 
                 simulation.width, simulation.textPressure, simulation.buttonStop, 
                 simulation.veloRatio, simulation.textTemperature]
        w = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))
        
        display(w)
        display(simulation.canvas)
        
    def updateValues():
            simulation.textVolume.value=simulation.width.value * simulation.height * 2 *simulation.radius
            simulation.textPressure.value = np.sum(simulation.impulse)/ (len(simulation.impulse)*simulation.dt * 4 *simulation.radius *(simulation.width.value +simulation.height))
            simulation.impulse = np.roll(simulation.impulse,1)
            simulation.impulse[0] = 0
            
            E_kin = 0
            if simulation.p:
                for i in simulation.p:
                    E_kin = E_kin+i.getKineticEnergy(simulation.dt)
                E_kin = E_kin/len(simulation.p)
                simulation.textTemperature.value=2/3 * E_kin/simulation.k
            
    def shutdown():
        simulation.startSimulation = False
        
    def step():
        with hold_canvas():
                simulation.canvas.clear()
                simulation.canvas.stroke_rect(0,0,width=simulation.width.value/10,height=simulation.height*10**12/10)
                for i in simulation.p:
                    simulation.canvas.fill_circle(i.x*10**12/10, i.y*10**12/10, i.r*10**12/10)
                    i.move(simulation.dt, simulation.veloRatio.value)
        simulation.collisionTest()
        simulation.updateValues()
        sleep(simulation.dt*10**12)
         

In [ ]:
simulation.deleteParticles()
simulation.initialize()

In [ ]:
simulation.shutdown()

In [ ]:
        dx = self.x-p.x
        dy = -(self.y-p.y)
        d = np.sqrt(dx**2+dy**2)
        alpha = np.arctan2(dy,dx)
        p1=np.array([self.x,self.y])
        p2=np.array([p.x,p.y])
        p1s=particle.rotMat(alpha,p1)
        p2s=particle.rotMat(alpha,p2)
        p1ns=np.array([p1s[0]+(2*self.r-d)/2,p1s[1]])
        p2ns=np.array([p2s[0]-(2*self.r-d)/2,p2s[1]])
        p1n=particle.rotMat(-alpha,p1ns)
        p2n=particle.rotMat(-alpha,p2ns)
        self.x=p1n[0]
        self.y=p1n[1]
        p.x=p2n[0]
        p.y=p2n[1]

In [ ]:
 p1=np.array([self.x,self.y])
        p2=np.array([p.x,p.y])
        p1s=particle.rotMat(alpha,p1)
        p2s=particle.rotMat(alpha,p2)
        p1ns=np.array([p1s[0]+(2*self.r-d)/2,p1s[1]])
        p2ns=np.array([p2s[0]-(2*self.r-d)/2,p2s[1]])
        p1n=particle.rotMat(-alpha,p1ns)
        p2n=particle.rotMat(-alpha,p2ns)
        self.x=p1n[0]
        self.y=p1n[1]
        p.x=p2n[0]
        p.y=p2n[1]